# Import Dependencies

In [1]:
from sklearn.model_selection import train_test_split
from sklearn import metrics
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Import Data

In [2]:
df = pd.read_csv('anomaly.csv')

In [3]:
df.head()

,Date,Power,Detector,Quality
0,1-Jan-16,96,8,Good
1,2-Jan-16,96,10,Good
2,3-Jan-16,91,8,Good
3,4-Jan-16,97,9,Good
4,5-Jan-16,91,11,Good


In [4]:
df.drop(['Date'], axis=1, inplace=True)

In [5]:
df.Quality.value_counts()

Good    1054
Bad      407
Name: Quality, dtype: int64

In [6]:
1054/len(df)

0.7214236824093087

In [7]:
df.Quality[df.Quality == 'Good'] = 1
df.Quality[df.Quality == 'Bad'] = 2

C:\Users\Rajeev\AppData\Local\Temp/ipykernel_33356/2335564477.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Quality[df.Quality == 'Good'] = 1
C:\Users\Rajeev\AppData\Local\Temp/ipykernel_33356/2335564477.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Quality[df.Quality == 'Bad'] = 2


In [8]:
good_mask = df['Quality']== 1 #All good to be True for good data points
bad_mask = df['Quality']== 2 #All values False for good data points
#print(good_mask.head())

df.drop('Quality',axis=1,inplace=True)

df_good = df[good_mask]
df_bad = df[bad_mask]

In [9]:
x_good = df_good.values
x_bad = df_bad.values

In [10]:
x_good_train, x_good_test = train_test_split(x_good, test_size=0.25, random_state=42)

# Model

In [11]:
model = Sequential()
model.add(Dense(10, input_dim=x_good.shape[1], activation='relu'))
model.add(Dense(3, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(x_good.shape[1])) 
model.compile(loss='mean_squared_error', optimizer='adam')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                30        
                                                                 
 dense_1 (Dense)             (None, 3)                 33        
                                                                 
 dense_2 (Dense)             (None, 10)                40        
                                                                 
 dense_3 (Dense)             (None, 2)                 22        
                                                                 
Total params: 125
Trainable params: 125
Non-trainable params: 0
_________________________________________________________________


In [12]:
model.fit(x_good_train,x_good_train,epochs=100)

Epoch 1/100
25/25 [==============================] - 2s 16ms/step - loss: 3813.3801
Epoch 2/100
25/25 [==============================] - 0s 6ms/step - loss: 3102.5996
Epoch 3/100
25/25 [==============================] - 0s 6ms/step - loss: 2315.7903
Epoch 4/100
25/25 [==============================] - 0s 6ms/step - loss: 1455.6934
Epoch 5/100
25/25 [==============================] - 0s 6ms/step - loss: 714.4618
Epoch 6/100
25/25 [==============================] - 0s 8ms/step - loss: 262.4648
Epoch 7/100
25/25 [==============================] - 0s 7ms/step - loss: 77.3248
Epoch 8/100
25/25 [==============================] - 0s 6ms/step - loss: 19.8184
Epoch 9/100
25/25 [==============================] - 0s 6ms/step - loss: 5.1639
Epoch 10/100
25/25 [==============================] - 0s 6ms/step - loss: 2.2683
Epoch 11/100
25/25 [==============================] - 0s 6ms/step - loss: 2.0111
Epoch 12/100
25/25 [==============================] - 0s 6ms/step - loss: 2.0019
Epoch 13/100
25/25

In [13]:
pred = model.predict(x_good_test)
score1 = np.sqrt(metrics.mean_squared_error(pred,x_good_test))

pred = model.predict(x_good)
score2 = np.sqrt(metrics.mean_squared_error(pred,x_good))

pred = model.predict(x_bad)
score3 = np.sqrt(metrics.mean_squared_error(pred,x_bad))

print(f"Insample Good Score (RMSE): {score1}".format(score1))
print(f"Out of Sample Good Score (RMSE): {score2}")
print(f"Bad sample Score (RMSE): {score3}")

Insample Good Score (RMSE): 1.244360619059351
Out of Sample Good Score (RMSE): 1.2434289487765253
Bad sample Score (RMSE): 3.0536329684915073


In [14]:
model.save('TensorFlow_Autoencoders_Anomaly_Detection')

INFO:tensorflow:Assets written to: TensorFlow_Autoencoders_Anomaly_Detection\assets
